In [ ]:
create_client = '''
CREATE OR REPLACE TABLE CLIENT (
    client_id  VARCHAR(50) PRIMARY KEY,
    email      VARCHAR(100) UNIQUE NOT NULL,
    name       VARCHAR(100) NOT NULL,
    cellphone  VARCHAR(20),
    address    VARCHAR(255),
    birth_date DATE NOT NULL
);
'''

In [ ]:
create_transaction = '''
CREATE  TABLE TRANSACTIONS (
    order_id  	VARCHAR(50) PRIMARY KEY,
    buyer_id    VARCHAR(100) NOT NULL,
    seller_id   VARCHAR(100) NOT NULL,
    created_at  DATETIME NOT NULL,
    item_id     VARCHAR(255) NOT NULL,
    category_id  VARCHAR(255) NOT NULL,
    condition	VARCHAR(255) NOT NULL,
  	shipping	VARCHAR(255) NOT NULL,
  	price		FLOAT	NOT NULL,
  	quantity	INTEGER	NOT NULL
);'''

In [ ]:
create_item = '''
CREATE TABLE ITEM (
    datetime_ref DATETIME NOT NULL,
    item_id      VARCHAR(50) NOT NULL,
    seller_id    VARCHAR(100) NOT NULL,
    published_at DATETIME NOT NULL,
    price        DECIMAL(10,2) NOT NULL,
    category_id  VARCHAR(50) NOT NULL,
    condition    VARCHAR(50) NOT NULL,
    shipping     VARCHAR(100) NOT NULL,
    PRIMARY KEY (datetime_ref, item_id, seller_id),
    FOREIGN KEY (category_id) REFERENCES CATEGORY(category_id),
    FOREIGN KEY (seller_id) REFERENCES CLIENT(client_id)
);
'''

In [ ]:
create_category = '''

CREATE TABLE CATEGORIA (
    category_id   INTEGER PRIMARY KEY AUTOINCREMENT, 
    category_name TEXT NOT NULL UNIQUE              
);

'''

In [ ]:
#CONSULTA PARA CLIENTES COM MAIS DE 1500 VENDAS EM JANEIRO/2020 E ANIVERSARIO HOJE

query = '''
        WITH 
        cte1 AS( 
            SELECT
                seller_id
            ,	created_at
            ,	COUNT(item_id) AS total_sells
            FROM	
                `order_table` 
            WHERE
                created_at BETWEEN "2020-01-01" AND "2020-01-31"
            GROUP BY
                seller_id, created_at
            HAVING	
                total_sells>1500
            ),
            
        cte2 AS(
            SELECT
                client_id
            ,	birth_date
            
            FROM
                `client_table` 
            WHERE	
                EXTRACT(MONTH FROM birth_date) = EXTRACT(MONTH FROM CURRENT_DATE) AND
                EXTRACT(DAY FROM birth_date) = EXTRACT(DAY FROM CURRENT_DATE)
            )
        
        SELECT
                cte1.seller_id
            ,	cte1.total_sells
                cte2.birth_date
        FROM	
                cte1
        LEFT JOIN 
                cte2
        ON
                cte1.seller_id = cte2.client_id;

'''

In [ ]:
query_v2 = '''
        WITH 
        cte1 AS( 
            SELECT
                seller_id,
            	created_at,
            	COUNT(item_id) AS total_sells
            FROM	
                TRANSACTIONS  
            WHERE
                created_at BETWEEN '2020-01-01' AND '2020-01-31'
            GROUP BY
                seller_id
            HAVING	
                total_sells>1500
            ),
            
        cte2 AS(
            SELECT
                client_id,
            	birth_date
            
            FROM
                CLIENT 
            WHERE	
              strftime('%m', birth_date) = strftime('%m', 'now') 
              AND strftime('%d', birth_date) = strftime('%d', 'now')
            )
        
        SELECT
			cte2.client_id,
            cte2.birth_date,
            cte1.total_sells

        FROM	
                cte2
        LEFT JOIN 
                cte1
        ON
                cte2.client_id = cte1.seller_id;

'''

In [ ]:
query_2 = '''
WITH 
	cte1 AS(
      SELECT	
      	category_id,
      	category_name
      FROM
      	CATEGORY
      WHERE
      	category_name = 'telemoveis'
      ),
      
    cte2 AS(
      SELECT 
      	seller_id,
      	category_id,
      	strftime('%Y-%m', created_at) AS year_month,
      	SUM(quantity) AS total_items_sold,
      	SUM(price * quantity) as sell_amount,
      	COUNT(order_id) AS total_sells
      FROM
      	TRANSACTIONS
      GROUP BY
      	seller_id,
      	year_month,
      	category_id
      ),
      
     cte3 AS(
       SELECT
       	cte1.category_id,
       	cte2.seller_id,
       	cte2.year_month,
       	cte2.total_items_sold,
       	cte2.sell_amount,
       	cte2.total_sells
       FROM
       	cte2
       INNER JOIN
       	cte1
       ON
       	cte1.category_id = cte2.category_id
     ),
     
     cte4 AS(
       SELECT
       	client_id,
       	name
       from 
       	CLIENT 
       )
 
 SELECT
 	cte3.category_id,
    cte4.name,
    cte3.seller_id,
    cte3.year_month,
    cte3.total_items_sold,
    cte3.sell_amount,
    cte3.total_sells
    
 FROM
 	cte3
 LEFT JOIN
 	cte4
 ON
 	cte3.seller_id = cte4.client_id

'''

In [ ]:
query_3 = '''
CREATE VIEW last_price_per_day AS
WITH ranked_prices AS (
    SELECT
        datetime_ref,
        item_id,
        seller_id,
        published_at,
        price,
        category_id,
        condition,
        shipping,
        ROW_NUMBER() OVER (PARTITION BY DATE(datetime_ref), item_id, seller_id ORDER BY datetime_ref DESC) AS rn
    FROM
        TRANSACTION 
SELECT
    datetime_ref,
    item_id,
    seller_id,
    published_at,
    price,
    category_id,
    condition,
    shipping
FROM
    ranked_prices
WHERE
    rn = 1;

'''